In [393]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score, auc, roc_curve
from sklearn.metrics import make_scorer
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.decomposition import PCA
import os

In [394]:
path_to_directory="AEEEM/"
files = [arff for arff in os.listdir(path_to_directory) if arff.endswith(".arff")]

def toCsv(content): 
    data = False
    header = ""
    newContent = []
    for line in content:
        if not data:
            if "@attribute" in line:
                attri = line.split()
                columnName = attri[attri.index("@attribute")+1]
                header = header + columnName + ","
            elif "@data" in line:
                data = True
                header = header[:-1]
                header += '\n'
                newContent.append(header)
        else:
            newContent.append(line)
    return newContent

In [395]:
# Main loop for reading and writing files
for zzzz,file in enumerate(files):
    with open(path_to_directory+file , "r") as inFile:
        content = inFile.readlines()
        name,ext = os.path.splitext(inFile.name)
        new = toCsv(content)
        with open(name+".csv", "w") as outFile:
            outFile.writelines(new)

In [396]:
eq = pd.read_csv('AEEEM/EQ.csv')
jdt = pd.read_csv('AEEEM/JDT.csv')
lc = pd.read_csv('AEEEM/LC.csv')
ml = pd.read_csv('AEEEM/ML.csv')
pde = pd.read_csv('AEEEM/PDE.csv')

In [397]:
list_relink = ['eq','jdt','lc','ml','pde']
list_data_relink = [eq,jdt,lc,ml,pde]

In [398]:
eq

,ck_oo_numberOfPrivateMethods,LDHH_lcom,LDHH_fanIn,numberOfNonTrivialBugsFoundUntil:,WCHU_numberOfPublicAttributes,WCHU_numberOfAttributes,CvsWEntropy,LDHH_numberOfPublicMethods,WCHU_fanIn,LDHH_numberOfPrivateAttributes,...,LDHH_fanOut,LDHH_numberOfMethodsInherited,LDHH_rfc,ck_oo_numberOfMethodsInherited,ck_oo_numberOfPublicMethods,LDHH_cbo,WCHU_numberOfLinesOfCode,CvsExpEntropy,LDHH_numberOfMethods,class
0,### ckooPrivateMethod LDHHlcom LDHHfanIn NNTBF...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,0.002547,0.002555,4.0,0.00,3.04,0.393707,0.003049,1.01,0.004091,...,0.005627,0.000000,0.004406,8.0,8.0,0.008431,3.50,0.103594,0.003611,buggy
2,37,0.008643,0.004756,71.0,0.00,14.37,2.093750,0.001481,2.02,0.015332,...,0.018761,0.001486,0.060301,7.0,7.0,0.021602,43.12,0.328692,0.009906,buggy
3,3,0.001479,0.009143,5.0,1.01,3.08,0.484675,0.000000,6.17,0.001953,...,0.003117,0.001486,0.002325,7.0,2.0,0.011859,4.68,0.125841,0.001655,clean
4,10,0.005642,0.005395,38.0,0.00,1.06,0.811584,0.000876,6.07,0.000739,...,0.020376,0.002338,0.030608,17.0,10.0,0.020478,24.06,0.170416,0.007000,buggy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,1,0.000000,0.000000,1.0,0.00,0.00,0.052387,0.000000,0.00,0.000000,...,0.004607,0.000000,0.000000,16.0,1.0,0.004539,1.04,0.041507,0.000000,clean
321,1,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,7.0,1.0,0.000000,0.00,0.000000,0.000000,clean
322,2,0.004633,0.018031,27.0,2.02,2.02,0.727254,0.003060,11.14,0.000000,...,0.011596,0.004215,0.018394,30.0,27.0,0.025180,23.19,0.267032,0.004671,buggy
323,0,0.000000,0.000000,1.0,0.00,0.00,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,9.0,2.0,0.000000,0.00,0.000000,0.000000,clean


In [399]:
for dt in list_data_relink:
    dt.drop(index=dt.index[0], axis=0, inplace=True)
    dt['Defective'] = dt['class']
    dt.loc[dt['Defective'] == 'clean', 'Defective'] = 'N'
    dt.loc[dt['Defective'] == 'buggy', 'Defective'] = 'Y'
    dt.drop('class',axis=1,inplace=True)

In [400]:
list_dataset = [pde]
label_dataset = ['pde']

list_test = [eq,jdt,lc,ml,pde]  
label_test = ['eq','jdt','lc','ml','pde'] 

my_index = range(0,len(list_test))
my_dataset = dict(zip(label_test,list_test))

df = list_dataset
df = pd.concat(df)
df

,ck_oo_numberOfPrivateMethods,LDHH_lcom,LDHH_fanIn,numberOfNonTrivialBugsFoundUntil:,WCHU_numberOfPublicAttributes,WCHU_numberOfAttributes,CvsWEntropy,LDHH_numberOfPublicMethods,WCHU_fanIn,LDHH_numberOfPrivateAttributes,...,LDHH_fanOut,LDHH_numberOfMethodsInherited,LDHH_rfc,ck_oo_numberOfMethodsInherited,ck_oo_numberOfPublicMethods,LDHH_cbo,WCHU_numberOfLinesOfCode,CvsExpEntropy,LDHH_numberOfMethods,Defective
1,0,0.003976,0.005189,2.0,0.00,3.03,0.105389,0.004203,2.04,0.004297,...,0.010649,0.002227,0.027707,14.0,6.0,0.008929,4.28,0.209245,0.004770,N
2,0,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,51.0,3.0,0.000000,0.00,0.000000,0.000000,N
3,8,0.002264,0.000000,7.0,0.00,0.00,0.032927,0.000000,0.00,0.000000,...,0.002006,0.000713,0.004680,14.0,2.0,0.001619,5.44,0.054240,0.002453,N
4,2,0.000000,0.001689,3.0,0.00,0.00,0.061022,0.000000,2.06,0.000000,...,0.001873,0.000000,0.005933,0.0,6.0,0.003134,6.39,0.046823,0.000000,Y
5,1,0.005692,0.007865,3.0,0.00,6.14,0.109004,0.001720,5.09,0.004365,...,0.002715,0.000000,0.007782,0.0,18.0,0.007727,8.66,0.051960,0.006210,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,16,0.007664,0.001443,8.0,0.00,0.00,0.056587,0.000000,1.01,0.000000,...,0.010813,0.000000,0.019267,0.0,2.0,0.013420,10.37,0.121457,0.007955,Y
1494,0,0.004142,0.000000,4.0,1.01,2.02,0.027205,0.004887,0.00,0.002642,...,0.002647,0.001324,0.007854,6.0,11.0,0.003189,3.13,0.190859,0.004892,N
1495,0,0.002679,0.004295,5.0,0.00,1.01,0.036016,0.002367,3.05,0.001510,...,0.001815,0.000000,0.003134,6.0,5.0,0.004879,3.41,0.065036,0.002969,N
1496,0,0.003033,0.011023,1.0,0.00,0.00,0.065500,0.003137,6.06,0.000000,...,0.000000,0.004833,0.003419,55.0,10.0,0.011873,3.93,0.050098,0.003455,N


In [401]:
def adjustdataset_list(list_df):
    dt_result = list_df.pop()
    
    for dt in list_df:
        if len(dt.columns) > len(dt_result.columns):
            dt_result = adjustdataset(dt,dt_result)
        else:
            dt_result = adjustdataset(dt_result,dt)    
    return dt_result

In [402]:
def adjustdataset(dt,dataframe):
    dt_copy = dt.copy()
    counter = 0
    
    for col in dataframe.columns:
        if not col in dt.columns:
            #print("Not Exist ",col)
            dt_copy.insert(counter,col,-1)
        counter += 1

    return pd.concat([dt_copy[dataframe.columns], dataframe])
    

In [403]:
def show_distribution(df, columns):
    
    counter = 0
    # Create a figure and a grid of subplots with 4 rows and 10 columns
    fig, axes = plt.subplots(5, 8, figsize=(20, 8))
    
    # Loop through each subplot and add content (optional)
    for i in range(5):
        for j in range(8):
            try:
                ax = axes[i, j]
                ax.set_title(columns[counter])
                ax.plot(df[columns[counter]].sort_values(), norm.pdf(df[columns[counter]].sort_values(), df[columns[counter]].mean(), df[columns[counter]].std())) 
                ax.set_xticks([])
                ax.set_yticks([])
                counter += 1
            except:
                pass

# Adjust layout to prevent overlapping titles/labels
plt.tight_layout()

# Display the plot
plt.show()

<Figure size 640x480 with 0 Axes>

In [404]:
def preprocessing(dataframe, imputer = None, pt = None, scaler = None, show = True):
    
    df = dataframe.copy()
    
    df.loc[(df['Defective'] == 'N'),'Defective']=0
    df.loc[(df['Defective'] == 'Y'),'Defective']=1
    
    change_dtypes = ['Defective']
    dict_dtypes = {d:'int64' for d in change_dtypes }
    df = df.astype(dict_dtypes)
    
    columns = df.columns
    
    if show:
        show_distribution(df,df.columns)
    
    if imputer :
        df = imputer.transform(df)
    else:
        imputer = KNNImputer(n_neighbors=2)
        df = imputer.fit_transform(df)
    
    df = pd.DataFrame(df)
    df.columns = columns
    
    if show:
        show_distribution(df,df.columns)
    
    y = df['Defective']
    X = df.drop('Defective',axis=1)
    
    if pt:
        X_norm = pt.transform(X)
    else:
        pt = PowerTransformer()
        X_norm = pt.fit_transform(X)
    
    X_norm = pd.DataFrame(X)
    X_norm.columns = X.columns
    
    if show:
        show_distribution(X_norm,X_norm.columns)

    if scaler:
        X_norm_scale = scaler.transform(X_norm)
    else:
        scaler = StandardScaler()
        X_norm_scale = scaler.fit_transform(X_norm)
        
    return pt, scaler, imputer, X_norm_scale, y



In [405]:
def heat_map(y,result,title):
    import seaborn as sns
    from sklearn.metrics import confusion_matrix
    sns.set
    _, ax = plt.subplots(figsize=(10,10))
    ax = sns.heatmap(confusion_matrix(y, result), annot=True, fmt='d', cmap= ['#09acec','#089dd9','#078fc5','#0681b1','#05668d','#05368d'], annot_kws={"size": 40, "weight": "bold"})  
    labels = ['No Defect','Defect']
    ax.set_xticklabels(labels, fontsize=30);
    ax.set_yticklabels(labels, fontsize=30,rotation=0);
    ax.set_ylabel('Prediction', fontsize=30,rotation=0);
    ax.set_xlabel('Ground Truth', fontsize=30) #0,1
    ax.set_title(title)

In [406]:
def score(y,result):
    accuracy = accuracy_score(y, result)
    f1 = f1_score(y, result, average='weighted')
    precision = precision_score(y, result, average='weighted')
    recall = recall_score(y, result, average='weighted')
    print("Accuracy = " , accuracy, " Precision = ",precision, " Recall = ",recall, " F1-Score = ",f1 )

In [407]:
from sklearn.metrics import make_scorer

scorers = {
            'f1_score': make_scorer(f1_score, average='micro'),
            'precision_score': make_scorer(precision_score, average='micro'),
            'recall_score': make_scorer(recall_score, average='micro'),
            'accuracy_score': make_scorer(accuracy_score)
          }
search_space={}

In [408]:
pt, scaler, imputer, X_norm_scale, y = preprocessing(df, show=False)
#X_norm_scale_smote, y_smote = X_norm_scale, y
smote = SMOTE(random_state=42)
X_norm_scale_smote, y_smote = smote.fit_resample(X_norm_scale, y)  

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [409]:
counter = 0
histories = []

lst_accu_stratified = []
lst_f1_stratified = []
lst_precision_stratified = []
lst_recall_stratified = []
lst_auc_stratified = []

lst_accu_stratified_val = []
lst_f1_stratified_val = []
lst_precision_stratified_val = []
lst_recall_stratified_val = []
lst_auc_stratified_val = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=19)

for train_index, test_index in skf.split(X_norm_scale_smote, y_smote):

    x_train_fold, x_test_fold = X_norm_scale_smote[train_index], X_norm_scale_smote[test_index]
    y_train_fold, y_test_fold = y_smote[train_index], y_smote[test_index]
 
    rte = RandomTreesEmbedding(n_estimators=1000, random_state=0, max_depth=10).fit(x_train_fold)
    x_sparse_embedding_train = rte.transform(x_train_fold)
    x_sparse_embedding_test = rte.transform(x_test_fold)
    
    with open('rte-tree-aeeem-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(rte, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    from sklearn.manifold import Isomap
    manifold = Isomap(n_components=100)
    x_sparse_manifold_train = manifold.fit_transform(x_sparse_embedding_train)
    x_sparse_manifold_test = manifold.transform(x_sparse_embedding_test)
    
    with open('manifold-tree-aeeem-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(manifold, handle, protocol=pickle.HIGHEST_PROTOCOL)

    scaler2 = StandardScaler()
    x_sparse_manifold_train = scaler2.fit_transform(x_sparse_manifold_train)
    x_sparse_manifold_test = scaler2.transform(x_sparse_manifold_test)
    
    with open('scaler-tree-aeeem-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(scaler2, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    input_X_train1 = layers.Input(
    shape=(x_sparse_manifold_train.shape[1],), name="X_train1")  
    
    input_X_train2 = layers.Input(shape=(x_train_fold.shape[1],), name="X_train2")

    layer_X_train1 = layers.Dense(64,activation='relu',name="X_train1_layer")(input_X_train1)
    layer_X_train1 = layers.BatchNormalization()(layer_X_train1)
    layer_X_train1 = layers.Dropout(0.3)(layer_X_train1)
    
    layer_X_train2 = layers.Dense(64,activation='relu',name="X_train2_layer")(input_X_train2)
    layer_X_train2 = layers.BatchNormalization()(layer_X_train2)
    layer_X_train2 = layers.Dropout(0.3)(layer_X_train2)
    
    # Merge all available features into a single large vector via concatenation
    concat = layers.concatenate([layer_X_train1, layer_X_train2])

    layer_final = layers.Dense(1, activation='sigmoid', name="classifier")(concat)
    sdp_classifier = Model(inputs=[input_X_train1, input_X_train2], outputs=layer_final)

    checkpointer=ModelCheckpoint(filepath='sdp-tree-aeeem-{0}.weights.best.keras'.format(counter),verbose=1,save_best_only=True)

    sdp_classifier.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'] )
    
    hist = sdp_classifier.fit([x_sparse_manifold_train,x_train_fold], y_train_fold, 
                    batch_size=32, epochs=100, validation_split= 0.1, callbacks=[checkpointer], 
                    verbose=2, shuffle=True)
    
    with open('sdp-classifier-tree-aeeem-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(sdp_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

    sdp_classifier.load_weights('sdp-tree-aeeem-{0}.weights.best.keras'.format(counter))

    histories.append(hist)

    results = sdp_classifier.predict([x_sparse_manifold_train,x_train_fold])
    results = np.round(results)
    
    lst_accu_stratified.append(accuracy_score(y_train_fold, results))
    lst_f1_stratified.append(f1_score(y_train_fold, results, average='weighted') )
    lst_precision_stratified.append(precision_score(results, y_train_fold, average='weighted') )
    lst_recall_stratified.append(recall_score(results, y_train_fold, average='weighted') )
    fpr, tpr, thresholds = roc_curve(y_train_fold, results)
    lst_auc_stratified.append(auc(fpr, tpr))
    
    results = sdp_classifier.predict([x_sparse_manifold_test,x_test_fold])
    results = np.round(results)
    
    lst_accu_stratified_val.append(accuracy_score(y_test_fold, results))
    lst_f1_stratified_val.append(f1_score(results, y_test_fold, average='weighted') )
    lst_precision_stratified_val.append(precision_score(results, y_test_fold, average='weighted') )
    lst_recall_stratified_val.append(recall_score(results, y_test_fold, average='weighted') )
    fpr, tpr, thresholds = roc_curve(y_test_fold, results)
    lst_auc_stratified_val.append(auc(fpr, tpr))

    print("Evaluate on training data")
    results = sdp_classifier.evaluate([x_sparse_manifold_train, x_train_fold], y_train_fold)
    print("test loss, test acc:", results)

    print("Evaluate on test data")
    results = sdp_classifier.evaluate([x_sparse_manifold_test , x_test_fold], y_test_fold)
    print("test loss, test acc:", results)
    counter += 1
    

dict_test = {  "train_accu":lst_accu_stratified, 
               "train_precision":lst_precision_stratified, "train_recall":lst_recall_stratified,
               "train_auc":lst_auc_stratified, "train_f1":lst_f1_stratified,
             
               "test_accu":lst_accu_stratified_val, 
               "test_precision":lst_precision_stratified_val, "test_recall":lst_recall_stratified_val, 
               "test_auc":lst_auc_stratified_val, "test_f1":lst_f1_stratified_val,
               "Input": x_train_fold.shape[1], "RTE": x_sparse_manifold_train.shape[1]}

dict_test_result = pd.DataFrame.from_dict(dict_test)
dict_test_result.to_csv("hasil.csv")

with open('histories-tree-manifold-aeeem.pickle'.format(counter), 'wb') as handle:
        pickle.dump(histories, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

Epoch 1/100

Epoch 1: val_loss improved from inf to 0.63219, saving model to sdp-tree-aeeem-0.weights.best.keras
66/66 - 2s - loss: 0.6759 - accuracy: 0.6611 - val_loss: 0.6322 - val_accuracy: 0.5948 - 2s/epoch - 24ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.63219 to 0.52709, saving model to sdp-tree-aeeem-0.weights.best.keras
66/66 - 0s - loss: 0.5269 - accuracy: 0.7589 - val_loss: 0.5271 - val_accuracy: 0.7069 - 194ms/epoch - 3ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.52709 to 0.45484, saving model to sdp-tree-aeeem-0.weights.best.keras
66/66 - 0s - loss: 0.4340 - accuracy: 0.7996 - val_loss: 0.4548 - val_accuracy: 0.7716 - 188ms/epoch - 3ms/step
Epoch 4/100

Epoch 4: val_loss improved from 0.45484 to 0.42981, saving model to sdp-tree-aeeem-0.weights.best.keras
66/66 - 0s - loss: 0.3919 - accuracy: 0.8289 - val_loss: 0.4298 - val_accuracy: 0.7974 - 191ms/epoch - 3ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.42981 to 0.41468, saving model to sdp-tree-

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.50702, saving model to sdp-tree-aeeem-1.weights.best.keras
66/66 - 2s - loss: 0.6555 - accuracy: 0.6798 - val_loss: 0.5070 - val_accuracy: 0.7414 - 2s/epoch - 24ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.50702 to 0.47418, saving model to sdp-tree-aeeem-1.weights.best.keras
66/66 - 0s - loss: 0.4711 - accuracy: 0.7943 - val_loss: 0.4742 - val_accuracy: 0.7888 - 201ms/epoch - 3ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.47418 to 0.45244, saving model to sdp-tree-aeeem-1.weights.best.keras
66/66 - 0s - loss: 0.4078 - accuracy: 0.8241 - val_loss: 0.4524 - val_accuracy: 0.7802 - 175ms/epoch - 3ms/step
Epoch 4/100

Epoch 4: val_loss improved from 0.45244 to 0.41352, saving model to sdp-tree-aeeem-1.weights.best.keras
66/66 - 0s - loss: 0.3755 - accuracy: 0.8375 - val_loss: 0.4135 - val_accuracy: 0.8060 - 180ms/epoch - 3ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.41352 to 0.38594, saving model to sdp-tree-

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.46174, saving model to sdp-tree-aeeem-2.weights.best.keras
66/66 - 2s - loss: 0.7330 - accuracy: 0.6347 - val_loss: 0.4617 - val_accuracy: 0.7845 - 2s/epoch - 25ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.46174 to 0.44674, saving model to sdp-tree-aeeem-2.weights.best.keras
66/66 - 0s - loss: 0.5103 - accuracy: 0.7646 - val_loss: 0.4467 - val_accuracy: 0.7888 - 194ms/epoch - 3ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.44674
66/66 - 0s - loss: 0.4416 - accuracy: 0.7987 - val_loss: 0.4581 - val_accuracy: 0.7672 - 127ms/epoch - 2ms/step
Epoch 4/100

Epoch 4: val_loss improved from 0.44674 to 0.43482, saving model to sdp-tree-aeeem-2.weights.best.keras
66/66 - 0s - loss: 0.3916 - accuracy: 0.8245 - val_loss: 0.4348 - val_accuracy: 0.7974 - 180ms/epoch - 3ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.43482 to 0.41287, saving model to sdp-tree-aeeem-2.weights.best.keras
66/66 - 0s - loss: 0.3582 - ac

...

Epoch 1/100

Epoch 1: val_loss improved from inf to 0.73684, saving model to sdp-tree-aeeem-7.weights.best.keras
66/66 - 2s - loss: 0.6980 - accuracy: 0.6713 - val_loss: 0.7368 - val_accuracy: 0.5129 - 2s/epoch - 25ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.73684 to 0.60202, saving model to sdp-tree-aeeem-7.weights.best.keras
66/66 - 0s - loss: 0.4813 - accuracy: 0.7782 - val_loss: 0.6020 - val_accuracy: 0.6422 - 182ms/epoch - 3ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.60202 to 0.49275, saving model to sdp-tree-aeeem-7.weights.best.keras
66/66 - 0s - loss: 0.4107 - accuracy: 0.8203 - val_loss: 0.4927 - val_accuracy: 0.7198 - 174ms/epoch - 3ms/step
Epoch 4/100

Epoch 4: val_loss improved from 0.49275 to 0.43068, saving model to sdp-tree-aeeem-7.weights.best.keras
66/66 - 0s - loss: 0.3729 - accuracy: 0.8285 - val_loss: 0.4307 - val_accuracy: 0.7802 - 185ms/epoch - 3ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.43068 to 0.37255, saving model to sdp-tree-

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 3 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.46730, saving model to sdp-tree-aeeem-8.weights.best.keras
66/66 - 2s - loss: 0.7076 - accuracy: 0.6550 - val_loss: 0.4673 - val_accuracy: 0.7026 - 2s/epoch - 24ms/step
Epoch 2/100

Epoch 2: val_loss did not improve from 0.46730
66/66 - 0s - loss: 0.5227 - accuracy: 0.7441 - val_loss: 0.4738 - val_accuracy: 0.7716 - 144ms/epoch - 2ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.46730 to 0.44442, saving model to sdp-tree-aeeem-8.weights.best.keras
66/66 - 0s - loss: 0.4537 - accuracy: 0.7992 - val_loss: 0.4444 - val_accuracy: 0.7716 - 190ms/epoch - 3ms/step
Epoch 4/100

Epoch 4: val_loss improved from 0.44442 to 0.43262, saving model to sdp-tree-aeeem-8.weights.best.keras
66/66 - 0s - loss: 0.4091 - accuracy: 0.8160 - val_loss: 0.4326 - val_accuracy: 0.7716 - 172ms/epoch - 3ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.43262 to 0.39934, saving model to sdp-tree-aeeem-8.weights.best.keras
66/66 - 0s - loss: 0.3729 - ac

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.60142, saving model to sdp-tree-aeeem-9.weights.best.keras
66/66 - 2s - loss: 0.6487 - accuracy: 0.6895 - val_loss: 0.6014 - val_accuracy: 0.6121 - 2s/epoch - 25ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.60142 to 0.49103, saving model to sdp-tree-aeeem-9.weights.best.keras
66/66 - 0s - loss: 0.4968 - accuracy: 0.7638 - val_loss: 0.4910 - val_accuracy: 0.7284 - 184ms/epoch - 3ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.49103 to 0.43295, saving model to sdp-tree-aeeem-9.weights.best.keras
66/66 - 0s - loss: 0.4302 - accuracy: 0.8079 - val_loss: 0.4330 - val_accuracy: 0.7802 - 170ms/epoch - 3ms/step
Epoch 4/100

Epoch 4: val_loss improved from 0.43295 to 0.38388, saving model to sdp-tree-aeeem-9.weights.best.keras
66/66 - 0s - loss: 0.3947 - accuracy: 0.8256 - val_loss: 0.3839 - val_accuracy: 0.8362 - 176ms/epoch - 3ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.38388 to 0.33311, saving model to sdp-tree-

In [410]:
dict_test_result

,train_accu,train_precision,train_recall,train_auc,train_f1,test_accu,test_precision,test_recall,test_auc,test_f1,Input,RTE
0,0.990940,0.990986,0.990940,0.990940,0.990940,0.922481,0.925485,0.922481,0.922481,0.922597,61,100
1,0.994392,0.994401,0.994392,0.994392,0.994392,0.906977,0.909981,0.906977,0.906977,0.907117,61,100
2,0.991372,0.991467,0.991372,0.991372,0.991371,0.875969,0.887988,0.875969,0.875969,0.876719,61,100
3,0.993529,0.993547,0.993529,0.993529,0.993529,0.906977,0.907097,0.906977,0.906977,0.906982,61,100
4,0.988783,0.988879,0.988783,0.988783,0.988783,0.910853,0.914488,0.910853,0.910853,0.911015,61,100
5,0.993097,0.993193,0.993097,0.993097,0.993097,0.922481,0.922601,0.922481,0.922481,0.922485,61,100
6,0.994825,0.994839,0.994825,0.994826,0.994825,0.891051,0.892121,0.891051,0.891140,0.891100,61,100
7,0.992669,0.992699,0.992669,0.992671,0.992669,0.910506,0.912934,0.910506,0.910641,0.910603,61,100
8,0.995257,0.995275,0.995257,0.995255,0.995257,0.894942,0.901803,0.894942,0.894713,0.895325,61,100
9,0.994394,0.994457,0.994394,0.994392,0.994394,0.902724,0.916140,0.902724,0.902404,0.903409,61,100


In [423]:
best = 8

dbfile = open('rte-tree-aeeem-{0}.pickle'.format(best), 'rb')    
rte = pickle.load(dbfile)
dbfile.close()

dbfile = open('manifold-tree-aeeem-{0}.pickle'.format(best), 'rb')    
manifold = pickle.load(dbfile)
dbfile.close()

dbfile = open('scaler-tree-aeeem-{0}.pickle'.format(best), 'rb')    
scaler2 = pickle.load(dbfile)
dbfile.close()

dbfile = open('sdp-classifier-tree-aeeem-{0}.pickle'.format(best), 'rb')    
sdp_classifier = pickle.load(dbfile)
dbfile.close()
sdp_classifier.load_weights('sdp-tree-aeeem-{0}.weights.best.keras'.format(best))

In [424]:
X_sparse_embedding = rte.transform(X_norm_scale)
X_sparse_manifold = manifold.transform(X_sparse_embedding)
X_sparse_manifold = scaler2.transform(X_sparse_manifold)
X_sparse_embedding.shape

(1497, 118018)

In [425]:
X_norm_scale.shape, df.shape

((1497, 61), (1497, 62))

In [426]:
print("Evaluate on training data")
results = sdp_classifier.evaluate([X_sparse_manifold, X_norm_scale], y, batch_size=128)
print("test loss, test acc:", results)

Evaluate on training data
12/12 [==============================] - 0s 2ms/step - loss: 0.0704 - accuracy: 0.9820
test loss, test acc: [0.07035829871892929, 0.9819639325141907]


In [427]:
from sklearn.metrics import classification_report

results = sdp_classifier.predict([X_sparse_manifold,X_norm_scale])
results=np.round(results)
print(classification_report(y,results))

47/47 [==============================] - 0s 937us/step
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99      1288
         1.0       0.90      0.98      0.94       209

    accuracy                           0.98      1497
   macro avg       0.95      0.98      0.96      1497
weighted avg       0.98      0.98      0.98      1497



In [428]:
print(y.shape, results.shape)
fpr, tpr, thresholds = roc_curve(y, results)
print(auc(fpr, tpr))

(1497,) (1497, 1)
0.9815020505809979


In [429]:
list_result = []

for label in my_dataset:
    dt = adjustdataset(my_dataset[label], df)
    pt, scaler, imputer, X_norm_scale_dt, y_dt = preprocessing(dt,imputer,pt, scaler, show=False)
    x_sparse_embedding_dt = rte.transform(X_norm_scale_dt)
    x_sparse_manifold_dt = manifold.transform(x_sparse_embedding_dt)
    x_sparse_manifold_dt = scaler2.transform(x_sparse_manifold_dt)
    results = sdp_classifier.predict([x_sparse_manifold_dt,X_norm_scale_dt])
    result  = np.round(results)
    fpr, tpr, thresholds = roc_curve(y_dt, result)
    list_result.append({'label':label, 'accuracy' : accuracy_score(y_dt, result),'precision':precision_score(y_dt, result, average='weighted'),
                  'recall': recall_score(y_dt, result, average='weighted'), 'auc':auc(fpr, tpr),
                        'f1-score':f1_score(y_dt, result, average='weighted')})
    #heat_map(y_dt,result,label)


    

94/94 [==============================] - 0s 1ms/step


In [430]:
df_result = pd.DataFrame(list_result)
df_result

,label,accuracy,precision,recall,auc,f1-score
0,eq,0.884130,0.889300,0.884130,0.828352,0.886290
1,jdt,0.894146,0.897667,0.894146,0.824647,0.895700
2,lc,0.932358,0.937889,0.932358,0.879697,0.934487
3,ml,0.867818,0.878756,0.867818,0.757252,0.872543
4,pde,0.981964,0.983204,0.981964,0.981502,0.982289


In [419]:
df_result[['accuracy','precision','recall','auc','f1-score']].mean()

accuracy     0.896234
precision    0.923975
recall       0.896234
auc          0.874335
f1-score     0.904801
dtype: float64

In [420]:
sdp_classifier.summary()

Model: "model_111"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 X_train1 (InputLayer)       [(None, 100)]                0         []                            
                                                                                                  
 X_train2 (InputLayer)       [(None, 61)]                 0         []                            
                                                                                                  
 X_train1_layer (Dense)      (None, 64)                   6464      ['X_train1[0][0]']            
                                                                                                  
 X_train2_layer (Dense)      (None, 64)                   3968      ['X_train2[0][0]']            
                                                                                          

In [421]:
list_result = []

for label in my_dataset:
    dt = my_dataset[label]
    list_result.append({'Dataset':label, 'Instances':dt.shape[0], 'Features':dt.shape[1],
                        'Defective Instances': dt[dt['Defective'] == 'Y'].shape[0],
                        'Non-Defective Instances': dt[dt['Defective'] == 'N'].shape[0],})


    

In [422]:
df_result = pd.DataFrame(list_result)
df_result

,Dataset,Instances,Features,Defective Instances,Non-Defective Instances
0,eq,324,62,129,195
1,jdt,997,62,206,791
2,lc,691,62,64,627
3,ml,1862,62,245,1617
4,pde,1497,62,209,1288
